# Section: Encrypted Deep Learning

- Lesson: Reviewing Additive Secret Sharing
- Lesson: Encrypted Subtraction and Public/Scalar Multiplication
- Lesson: Encrypted Computation in PySyft
- Project: Build an Encrypted Database
- Lesson: Encrypted Deep Learning in PyTorch
- Lesson: Encrypted Deep Learning in Keras
- Final Project

# Lesson: Reviewing Additive Secret Sharing

_For more great information about SMPC protocols like this one, visit https://mortendahl.github.io. With permission, Morten's work directly inspired this first teaching segment._

In [1]:
import random
import numpy as np

BASE = 10

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

def encrypt(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

def decrypt(sharing):
    return sum(sharing) % Q

def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [2]:
x = encrypt(encode(5.5))
x

[200586597334110239648081182858842528589,
 204926951123661908084131222454606661440,
 182433142492562346408678150247832298797]

In [3]:
y = encrypt(encode(2.3))
y

[205360498566252727056817627142307939734,
 215983483300439481912404806303402943740,
 166602709083642285171668122115250605351]

In [4]:
z = add(x,y)
z

(111973750425195719634453532220784723910,
 126937088948934142926090750977643860767,
 55062506101037384509900994582717159735)

In [5]:
decode(decrypt(z))

7.79999999

# Lesson: Encrypted Subtraction and Public/Scalar Multiplication

In [6]:
field = 23740629843760239486723

In [7]:
x = 5

bob_x_share = 2372385723 # random number
alices_x_share = field - bob_x_share + x

In [8]:
(bob_x_share + alices_x_share) % field

5

In [9]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [10]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field

4

In [11]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field

4

In [12]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share

26

In [13]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [14]:
(bob_z_share + alice_z_share) % field

4

In [15]:
def sub(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [16]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [17]:
bob_x_share + alice_x_share

15

In [18]:
bob_y_share + alice_y_share

11

In [19]:
((bob_y_share * 3) + (alice_y_share * 3)) % field

3

In [20]:
def imul(a, scalar):
    
    # logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [21]:
x = encrypt(encode(5.5))
x

[35588733695667958677668760486088147387,
 156775780026187286915441394721797619641,
 101608831753312001477335122573029977385]

In [22]:
z = imul(x, 3)

In [23]:
decode(decrypt(z))

16.5

# Lesson: Encrypted Computation in PySyft

In [24]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

In [25]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [26]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [27]:
x = x.share(bob, alice, crypto_provider=secure_worker)

In [28]:
y = y.share(bob, alice, crypto_provider=secure_worker)

In [29]:
z = x + y
z.get()

tensor([3, 1, 4, 4])

In [30]:
z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [31]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [32]:
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [33]:
z = x < y
z.get()

tensor([1, 0, 0, 0])

In [34]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [35]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [36]:
z = x + y
z.get().float_precision()

tensor([3., 1., 4., 4.])

In [37]:
z = x - y
z.get().float_precision()

tensor([-1.,  3.,  2.,  4.])

In [38]:
z = x * y
z.get().float_precision()

tensor([ 2., -2.,  3.,  0.])

In [39]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [40]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [41]:
z = x == y
z.get().float_precision()

tensor([0., 0., 0., 0.])

# Project: Build an Encrypted Database

In [ ]:
# try this project here!

In [73]:
import string
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

In [74]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [75]:
char2index = {}
index2char = {}

In [76]:
for i,char in enumerate(' ' + string.ascii_lowercase + '0123456789' + string.punctuation):
    char2index[char] = i
    index2char[i] = char

In [77]:
def string2values(str_input, max_len=8):
    
    str_input = str_input[:max_len].lower()
    
    #pad short strings
    if(len(str_input) < max_len):
        str_input = str_input + "." * (max_len - len(str_input))
        
    values = list()
    for char in str_input:
        values.append(char2index[char])
        
    return th.tensor(values).long()

def values2string(input_values):
    
    s = ""
    
    for value in input_values:
        s += index2char[int(value)]
    
    return s

def strings_equal(str_a, str_b):
    
    vect = (str_a * str_b).sum(1)
    
    x = vect[0]
    
    for i in range(vect.shape[0] - 1):
        x = x * vect[i + 1]
        
    key_match = x
    return key_match

def one_hot(index, length):
    vect = th.zeros(length).long()
    vect[index] = 1
    return vect

def string2one_hot_matrix(str_input, max_len=8):
    
    str_input = str_input[:max_len].lower()
    
    #pad short strings
    if(len(str_input) < max_len):
        str_input = str_input + "." * (max_len - len(str_input))
        
    char_vectors = list()
    for char in str_input:
        char_v = one_hot(char2index[char], len(index2char)).unsqueeze(0)
        char_vectors.append(char_v)
        
    return th.cat(char_vectors, dim=0)

In [82]:
class EncryptedDB():
    
    def __init__(self, *owners, max_key_len=8, max_val_len=8):
        self.max_key_len = max_key_len
        self.max_val_len = max_val_len
        
        self.keys = list()
        self.values = list()
        self.owners = owners
        
    def add_entry(self, key, value):
        key = string2one_hot_matrix(key)
        key = key.share(*self.owners)
        self.keys.append(key)
        
        value = string2one_hot_matrix(value)
        value = value.share(*self.owners)
        self.values.append(value)
        
    def query(self, query_str):
        query_matrix = string2one_hot_matrix(query_str)
        
        query_matrix = query_matrix.share(*self.owners)

        key_matches = list()
        for key in self.keys:
            key_match = strings_equal(key, query_matrix)
            key_matches.append(key_match)

        result = self.values[0] * key_matches[0]

        for i in range(len(self.values) - 1):
            result += self.values[i+1] * key_matches[i+1]
            
        result = result.get()

        return values2string(result).replace(".", "")
        

In [83]:
db = EncryptedDB(bob, alice, secure_worker)

db.add_entry("key1", "value1")
db.add_entry("key2", "value2")
db.add_entry("key3", "value3")
db.add_entry("key4", "value4")

In [84]:
db.query("key1")

ValueError: only one element tensors can be converted to Python scalars

# Lesson: Encrypted Deep Learning in PyTorch

### Train a Model

In [2]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

W0708 19:08:54.417259 4524656064 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0708 19:08:54.431255 4524656064 deprecation_wrapper.py:119] From /anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



tensor(1.5446)
tensor(1.1046)
tensor(0.9709)
tensor(0.6789)
tensor(0.6226)
tensor(0.3577)
tensor(0.2948)
tensor(0.1633)
tensor(0.1226)
tensor(0.0737)
tensor(0.0581)
tensor(0.0382)
tensor(0.0295)
tensor(0.0201)
tensor(0.0151)
tensor(0.0104)
tensor(0.0080)
tensor(0.0057)
tensor(0.0044)
tensor(0.0033)


In [3]:
model(data)

tensor([[-0.0290],
        [-0.0034],
        [ 0.9915],
        [ 0.9604]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [11]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

W0708 19:17:51.995477 4524656064 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0708 19:17:51.996447 4524656064 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0708 19:17:51.997471 4524656064 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior


In [13]:
encrypted_model = model.share(alice, bob, crypto_provider=secure_worker)

In [14]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:68575856941 -> alice:11700938997]
 	-> (Wrapper)>[PointerTensor | me:36938778567 -> bob:5345461600]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:34106917749 -> alice:17820762473]
 	-> (Wrapper)>[PointerTensor | me:63186202512 -> bob:59077679185]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:48152303773 -> alice:24740687114]
 	-> (Wrapper)>[PointerTensor | me:52282682167 -> bob:88135850039]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:64262426897 -> alice:94534438255]
 	-> (Wrapper)>[PointerTensor | me:41685855345 -> bob:723686652

In [15]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [16]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:20628047478 -> alice:63898619451]
	-> (Wrapper)>[PointerTensor | me:55560150734 -> bob:86655416511]
	*crypto provider: secure_worker*

In [20]:
encrypted_prediction = encrypted_model(encrypted_data)

In [21]:
encrypted_prediction.get().float_precision()

tensor([[-0.0280],
        [-0.0030],
        [ 0.9900],
        [ 0.9590]])

# Lesson: Encrypted Deep Learning in Keras


## Step 1: Public Training

Welcome to this tutorial! In the following notebooks you will learn how to provide private predictions. By private predictions, we mean that the data is constantly encrypted throughout the entire process. At no point is the user sharing raw data, only encrypted (that is, secret shared) data. In order to provide these private predictions, Syft Keras uses a library called [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) under the hood. TF Encrypted combines cutting-edge cryptographic and machine learning techniques, but you don't have to worry about this and can focus on your machine learning application.

You can start serving private predictions with only three steps:
- **Step 1**: train your model with normal Keras.
- **Step 2**: secure and serve your machine learning model (server).
- **Step 3**: query the secured model to receive private predictions (client). 

Alright, let's go through these three steps so you can deploy impactful machine learning services without sacrificing user privacy or model security.

Huge shoutout to the Dropout Labs ([@dropoutlabs](https://twitter.com/dropoutlabs)) and TF Encrypted ([@tf_encrypted](https://twitter.com/tf_encrypted)) teams for their great work which makes this demo possible, especially: Jason Mancuso ([@jvmancuso](https://twitter.com/jvmancuso)), Yann Dupis ([@YannDupis](https://twitter.com/YannDupis)), and Morten Dahl ([@mortendahlcs](https://github.com/mortendahlcs)). 

_Demo Ref: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials_

## Train Your Model in Keras

To use privacy-preserving machine learning techniques for your projects you should not have to learn a new machine learning framework. If you have basic [Keras](https://keras.io/) knowledge, you can start using these techniques with Syft Keras. If you have never used Keras before, you can learn a bit more about it through the [Keras documentation](https://keras.io). 

Before serving private predictions, the first step is to train your model with normal Keras. As an example, we will train a model to classify handwritten digits. To train this model we will use the canonical [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

We borrow [this example](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) from the reference Keras repository.  To train your classification model, you just run the cell below.

In [22]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

11493376/11490434 [==============================] - 1s 0us/step


W0708 19:23:50.866875 4524656064 deprecation.py:506] From /anaconda3/envs/pysyft/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 7s 112us/sample - loss: 2.3011 - acc: 0.1388 - val_loss: 2.3006 - val_acc: 0.1549
Epoch 2/2
60000/60000 [==============================] - 7s 115us/sample - loss: 2.3000 - acc: 0.1538 - val_loss: 2.2995 - val_acc: 0.1672
Test loss: 2.2994820541381835
Test accuracy: 0.1672


In [23]:
## Save your model's weights for future private prediction
model.save('short-conv-mnist.h5')

## Step 2: Load and Serve the Model

Now that you have a trained model with normal Keras, you are ready to serve some private predictions. We can do that using Syft Keras.

To secure and serve this model, we will need three TFEWorkers (servers). This is because TF Encrypted under the hood uses an encryption technique called [multi-party computation (MPC)](https://en.wikipedia.org/wiki/Secure_multi-party_computation). The idea is to split the model weights and input data into shares, then send a share of each value to the different servers. The key property is that if you look at the share on one server, it reveals nothing about the original value (input data or model weights).

We'll define a Syft Keras model like we did in the previous notebook. However, there is a trick: before instantiating this model, we'll run `hook = sy.KerasHook(tf.keras)`. This will add three important new methods to the Keras Sequential class:
 - `share`: will secure your model via secret sharing; by default, it will use the SecureNN protocol from TF Encrypted to secret share your model between each of the three TFEWorkers. Most importantly, this will add the capability of providing predictions on encrypted data.
 - `serve`: this function will launch a serving queue, so that the TFEWorkers can can accept prediction requests on the secured model from external clients.
 - `shutdown_workers`: once you are done providing private predictions, you can shut down your model by running this function. It will direct you to shutdown the server processes manually if you've opted to manually manage each worker.

If you want learn more about MPC, you can read this excellent [blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In [24]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

## Model

As you can see, we define almost the exact same model as before, except we provide a `batch_input_shape`. This allows TF Encrypted to better optimize the secure computations via predefined tensor shapes. For this MNIST demo, we'll send input data with the shape of (1, 28, 28, 1). 
We also return the logit instead of softmax because this operation is complex to perform using MPC, and we don't need it to serve prediction requests.

In [25]:
num_classes = 10
input_shape = (1, 28, 28, 1)

model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

### Load Pre-trained Weights

With `load_weights` you can easily load the weights you have saved previously after training your model.

In [26]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

## Step 3: Setup Your Worker Connectors

Let's now connect to the TFEWorkers (`alice`, `bob`, and `carol`) required by TF Encrypted to perform private predictions. For each TFEWorker, you just have to specify a host.

These workers run a [TensorFlow server](https://www.tensorflow.org/api_docs/python/tf/distribute/Server), which you can either manage manually (`AUTO = False`) or ask the workers to manage for you (`AUTO = True`). If choosing to manually manage them, you will be instructed to execute a terminal command on each worker's host device after calling `model.share()` below.  If all workers are hosted on a single device (e.g. `localhost`), you can choose to have Syft automatically manage the worker's TensorFlow server.

In [27]:
AUTO = False

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

## Step 4: Split the Model Into Shares

Thanks to `sy.KerasHook(tf.keras)` you can call the `share` method to transform your model into a TF Encrypted Keras model.

If you have asked to manually manage servers above then this step will not complete until they have all been launched. Note that your firewall may ask for Python to accept incoming connection.

In [ ]:
model.share(alice, bob, carol)

W0708 19:24:33.277317 4524656064 deprecation_wrapper.py:119] From /anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/keras/engine/base_layer_utils.py:29: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0708 19:24:33.278790 4524656064 deprecation_wrapper.py:119] From /anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/tensor/native.py:403: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0708 19:24:33.284943 4524656064 deprecation_wrapper.py:119] From /anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/tensor/native.py:101: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0708 19:24:33.340935 4524656064 deprecation.py:506] From /anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/tensor/shared.py:62: calling extract_image_patches (from tensorflow.python.ops.array_ops) with ksizes is deprecated and will be removed in a fut

## Step 5: Launch 3 Servers

```
python -m tf_encrypted.player --config /tmp/tfe.config server0
python -m tf_encrypted.player --config /tmp/tfe.config server1
python -m tf_encrypted.player --config /tmp/tfe.config server2```

## Step 6: Serve the Model

Perfect! Now by calling `model.serve`, your model is ready to provide some private predictions. You can set `num_requests` to set a limit on the number of predictions requests served by the model; if not specified then the model will be served until interrupted.

In [ ]:
model.serve(num_requests=3)

## Step 7: Run the Client

At this point open up and run the companion notebook: Section 4b - Encrytped Keras Client

## Step 8: Shutdown the Servers

Once your request limit above, the model will no longer be available for serving requests, but it's still secret shared between the three workers above. You can kill the workers by executing the cell below.

**Congratulations** on finishing Part 12: Secure Classification with Syft Keras and TFE!

In [ ]:
model.shutdown_workers()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config /tmp/tfe.config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))

# Keystone Project - Mix and Match What You've Learned

Description: Take two of the concepts you've learned about in this course (Encrypted Computation, Federated Learning, Differential Privacy) and combine them for a use case of your own design. Extra credit if you can get your demo working with [WebSocketWorkers](https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials/advanced/websockets-example-MNIST) instead of VirtualWorkers! Then take your demo or example application, write a blogpost, and share that blogpost in #general-discussion on OpenMined's slack!!!

Inspiration:
- This Course's Code: https://github.com/Udacity/private-ai
- OpenMined's Tutorials: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials
- OpenMined's Blog: https://blog.openmined.org